# Time 8
## Preparação de dados

Montagem do dataset.

Compomos um conjunto de dados a partir das combinações de formas de pagamentos distintas que ocorrem no Banco, considerando o pagamento com cartão CSF, cartões de terceiros e "outras formas".

In [48]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import logging
import os
import cx_Oracle
import pandas as pd

#logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
#from ads.dataset.factory import DatasetFactory
#from sqlalchemy import create_engine

# Add TNS_ADMIN to the environment
os.environ['TNS_ADMIN'] = "/home/datascience/ADB"
connection = cx_Oracle.connect('hackathon', 'bCfrCC0981267', 'adw_low')

df_CPFs é o dataframe que contém os CPFs com sua respectiva forma de pagamento. -1 significa que não tem forma atribuída.
Carregamos cerca de 3 mil CPFs distintos do mês com menos compras (junho/2020) que possuem compras nos 5 departamentos selecionados

In [49]:
df_CPFs = pd.read_sql("SELECT * FROM (SELECT DISTINCT CPF_CRIP CPF, -1 AS FORMA from EVE_BUFFER_CCI_MANU_HCKT WHERE ANOMES = '202006' AND PRF_DS_DEPARTAMENTO IN ('PGC', 'PERECIVEIS', 'ELETRO', 'BAZAR')) WHERE ROWNUM <= 40000", con=connection)

df_pagamentos contém os dados sobre departamentos, formas de pagamentos e frenquências

In [62]:
df_pagamentos = pd.read_csv('regras_parcelamento12d.csv', delimiter='\t')
df_pagamentos

,ds_departament,qty_num_parcel,CARTAO CARREFOUR,TEF,Frequencia,QTDE,ID
0,PGC,2,0.606920,0.393080,0.345498,864,1
1,PERECIVEIS,2,0.610032,0.389968,0.138891,347,2
2,ELETRO,2,0.389690,0.610310,0.131207,328,3
3,PGC,1,0.166819,0.833181,0.074287,185,4
4,BAZAR,2,0.396687,0.603313,0.067643,169,5
5,ELETRO,30,1.000000,0.000000,0.045158,113,6
6,TEXTIL,2,0.537017,0.462983,0.032402,81,7
7,PERECIVEIS,1,0.166157,0.833843,0.029168,729,8
8,ELETRO,1,0.423205,0.576795,0.026826,67,9
9,ELETRO,13,1.000000,0.000000,0.019079,48,10


Dataframe df_CPFs atualizado, tendo uma forma de pagamento atribuída a cada CPF

In [51]:
import time

count_cpfs = 0
ultimo_count_cpf = 0

start_time = time.time()

for index, pagamento in df_pagamentos.iterrows():
    qtd_cpfs = pagamento['QTDE']
    count_cpfs = ultimo_count_cpf
    
    start_time = time.time()
    while qtd_cpfs > 0:
        while df_CPFs.loc[count_cpfs, 'FORMA'] != -1:
            count_cpfs += 1
        
        query = "SELECT cpf_crip FROM EVE_BUFFER_CCI_MANU_HCKT WHERE cpf_crip = "
        query += f"\'{df_CPFs.loc[count_cpfs, 'CPF']}\'"
        query += " and prf_ds_departamento = "
        query += f"\'{pagamento['ds_departament']}\'"
        query += " AND ANOMES = '202006' AND ROWNUM = 1"
        
        #print(query)
        #print(count_cpfs)
        if qtd_cpfs % 100 == 0:
            print(qtd_cpfs)
            print("--- %s seconds ---" % (time.time() - start_time))
          
        result = pd.read_sql(query, con = connection)
        
        if len(result) > 0:
            df_CPFs.loc[count_cpfs, 'FORMA'] = pagamento['ID']
            qtd_cpfs -= 1
        
        count_cpfs += 1
        ultimo_count_cpf = count_cpfs

print("--- %s seconds ---" % (time.time() - start_time))

df_CPFs.to_csv(r'saida.csv', index = False)
#df_CPFs

800
--- 8.492346286773682 seconds ---
800
--- 8.569202661514282 seconds ---
700
--- 20.197595357894897 seconds ---
600
--- 31.68448567390442 seconds ---
500
--- 43.69326138496399 seconds ---
400
--- 58.19095802307129 seconds ---
300
--- 70.96685886383057 seconds ---
200
--- 83.17573714256287 seconds ---
100
--- 98.29430341720581 seconds ---
100
--- 98.61033058166504 seconds ---
300
--- 7.737156867980957 seconds ---
300
--- 7.8116960525512695 seconds ---
200
--- 23.919963598251343 seconds ---
100
--- 38.40088677406311 seconds ---
100
--- 38.48019361495972 seconds ---
300
--- 9.537278413772583 seconds ---
200
--- 49.87904906272888 seconds ---
200
--- 49.979326009750366 seconds ---
200
--- 50.026785373687744 seconds ---
200
--- 50.093339681625366 seconds ---
200
--- 50.19758653640747 seconds ---
200
--- 50.255754232406616 seconds ---
200
--- 50.40074062347412 seconds ---
200
--- 50.443331480026245 seconds ---
200
--- 50.48354458808899 seconds ---
200
--- 50.606117248535156 seconds ---
200

,CPF,FORMA
0,00002CF143F,1
1,0000368E069,1
2,00010BB56B4,1
3,00016C740C9,1
4,0001AB200CB,1
5,000201E042E,1
6,0002224092D,1
7,0002244CD2C,1
8,0002437132C,-1
9,0002539B574,1


Montado o dataframe de compras, carregando todas as compras nos 5 departamentos selecionados desses CPFs, com o valor totalizado por CPF e departamento

In [85]:
q_COMPRAS = '''
SELECT CPF_CRIP CPF, PRF_DS_DEPARTAMENTO, sum(TICU_VL_TOTALITEM) as VALOR
FROM EVE_BUFFER_CCI_MANU_HCKT 
WHERE CPF_CRIP IN (
    select * from (
        SELECT DISTINCT CPF_CRIP CPF 
        from EVE_BUFFER_CCI_MANU_HCKT 
        WHERE ANOMES = '202006' AND PRF_DS_DEPARTAMENTO IN ('PGC', 'PERECIVEIS', 'ELETRO', 'BAZAR', 'TEXTIL')
    ) where ROWNUM <= 25000)
AND PRF_DS_DEPARTAMENTO IN ('PGC', 'PERECIVEIS', 'ELETRO', 'BAZAR', 'TEXTIL')
GROUP BY CPF_CRIP, PRF_DS_DEPARTAMENTO
ORDER BY PRF_DS_DEPARTAMENTO
'''
df_COMPRAS = pd.read_sql(q_COMPRAS, con=connection)
len(df_COMPRAS)
df_COMPRAS.to_csv(r'1_compras.csv', index = False)

Juntando os dados de compras com as formas de pagamento.

In [86]:
df_compras_formas = pd.merge(df_COMPRAS, df_CPFs, on='CPF')
len(df_compras_formas)
df_compras_formas.to_csv(r'2_compras_formas.csv', index = False)

Dataframe com a junção das compras e formas com os pagamentos

In [87]:
df_compras_formas_e_pagamentos = pd.merge(df_compras_formas, df_pagamentos.rename(columns={"ID": "FORMA"}), how='inner', left_on=['FORMA', 'PRF_DS_DEPARTAMENTO'], right_on=['FORMA', 'ds_departament'])
len(df_compras_formas_e_pagamentos)

df_compras_formas_e_pagamentos.to_csv(r'3_compras_formas_e_pagamentos.csv', index = False)

Criação das colunas calculadas com as proporções.

In [88]:
df_compras_formas_e_pagamentos['percentual_1'] = round(df_compras_formas_e_pagamentos['VALOR']*df_compras_formas_e_pagamentos['CARTAO CARREFOUR'], 2)
df_compras_formas_e_pagamentos['percentual_2'] = round(df_compras_formas_e_pagamentos['VALOR']*df_compras_formas_e_pagamentos['TEF'], 2)
df_compras_formas_e_pagamentos

df_compras_formas_e_pagamentos.to_csv(r'4_compras_formas_e_pagamentos_com_percentuais.csv', index = False)

In [90]:
df_final = df_compras_formas_e_pagamentos.groupby(['PRF_DS_DEPARTAMENTO', 'FORMA']).count()

depto_forma_total = df_compras_formas_e_pagamentos[['FORMA','ds_departament','qty_num_parcel','percentual_1', 'percentual_2']].groupby(['FORMA', 'ds_departament', 'qty_num_parcel']).sum()
depto_forma_total.sort_values(by=['ds_departament', 'qty_num_parcel'])

depto_forma_total.to_csv(r'5_depto_forma_total.csv', index = True)
depto_forma_total

,,,percentual_1,percentual_2
FORMA,ds_departament,qty_num_parcel,,
1,PGC,2,346001.19,224092.08
2,PERECIVEIS,2,74780.83,47804.26
3,ELETRO,2,240503.87,376663.29
4,PGC,1,21035.30,105061.52
5,BAZAR,2,18462.84,28079.71
6,ELETRO,30,188717.98,0.00
7,TEXTIL,2,8185.05,7056.69
8,PERECIVEIS,1,44793.21,224790.95
9,ELETRO,1,51788.14,70583.20


Enriquecendo o dataset acima com os dados de custos, temos que:

In [94]:
df_depto_forma_total_custos = pd.read_csv('6_depto_forma_total_custos.csv', delimiter=';')
df_depto_forma_total_custos

,FORMA,ds_departament,qty_num_parcel,percentual_1,percentual_2,Interchange,Fator,Custo_1,Custo_2,Custo_total
0,1,PGC,2,"346001,189999999","224092,079999999","1,500%","1,0169983","-5696,3817417958","-3689,3342274142","-9385,7159692099"
1,2,PERECIVEIS,2,"74780,83","47804,2599999999","1,500%","1,0169983","-1231,1522820148","-787,0242118071","-2018,1764938219"
2,3,ELETRO,2,"240503,869999999","376663,289999999","1,500%","1,0169983","-3959,5293123102","-6201,1864408926","-10160,7157532029"
3,4,PGC,1,"21035,2999999999","105061,519999999","1,500%","1,011028","-226,0052432514","-1128,7908603141","-1354,7961035655"
4,5,BAZAR,2,"18462,84","28079,71","1,500%","1,0169983","-303,9624941108","-462,2895873824","-766,2520814932"
5,6,ELETRO,30,"188717,98",0,"0,900%","1,0752548","-13089,0989156545",0,"-13089,0989156545"
6,7,TEXTIL,2,"8185,05","7056,69","1,500%","1,0169983","-134,7543613237","-116,1776353241","-250,9319966478"
7,8,PERECIVEIS,1,"44793,21","224790,949999999","1,500%","1,011028","-481,2624646219","-2415,1751263575","-2896,4375909795"
8,9,ELETRO,1,"51788,14","70583,2","1,500%","1,011028","-556,4166509743","-758,352544792","-1314,7691957663"
9,10,ELETRO,13,"96690,9599999999",0,"1,800%","1,0429417","-3909,458085227",0,"-3909,458085227"
